# Fine-Tuning Neural Network Hyperparameters

There are many hyperparameters to tune in neural networks like number of layers, number of units in each layer, activation function in each layer, etc.

One way to find a combination is to use Sklearn's `GridSearchCV` or `RandomizedSearchCV` to explore the hyperparameter space.

So we need to wrap Keras model to Sklearn model.So let's create a function that build and compile a model, given a set of hyperparameters. And then convert this model using `KerasRegressor`.

In [1]:
import tensorflow
from tensorflow import keras

In [10]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

In [11]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

Now we can use it like Sklearn regressor also it works as Keras model.

In [12]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [13]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [14]:
keras_reg.fit(X_train, y_train, epochs=20,
             validation_data=(X_val, y_val))
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_test[:3])

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 1.9551 - val_loss: 0.7994
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.7938 - val_loss: 0.6934
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6655 - val_loss: 0.6336
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6336 - val_loss: 0.5937
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5824 - val_loss: 0.5666
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5831 - val_loss: 0.5708
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5355 - val_loss: 0.5299
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5180 - val_loss: 0.5185
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5104 - val_loss: 0.5069
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4914 - val_loss: 0.4950

Note that any extra parameter you pass to `fit()` method will get passed to the underlying Keras model. 

Now, we want to train it on many variants to see which combination of hyperparameters works best for validation set. We use randomized search because there are many hyperparameters:

```python
import numpy as np
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    'n_hidden': [0, 1, 2, 3],
    'n_neurons': np.arange(1, 100),
    'learning_rate': reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RanomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train)
```

Note that `RandomizedSearchCV` use K-fold cross-validation, so it does not use `X_val` and `y_val`. Since, Sklearn regression `score()` is opposite to the Keras `evaluate()`.

This training will take several hours, depending on the hardware, size of dataset, complexity of model, and values of `n_iter` and `cv`. After this done you can check best hyperparameters by `rnd_search_cv.best_params_`, score by `rnd_search_cv.best_score_`, and model by `rnd_search_cv.best_estimator_.model`.

There are many techniques to explore search space much more efficiently than randomly. The main idea is: when a region is turns out to be good, then it should be more explored. This is called "zooming".

## Number of Hidden Layers

Usually you can begin with single hidden layer for many problems. But for complex problems, deep networks have a much higher *parameter efficiency*. As in Deep Networks, lower hidden layers model lower structures (line, orientation), then higher hidden layers combine those to learn complex patterns. Thus adding more layers is beneficial.

DNN also helps in generalize to new datasets. Instead of randomly initialize you could use pretrained lower layers of similar tasks. This is called *Transfer Learning*.

So you can get good results with two-three hidden layers. For more complex layers you can increase number of layers until model overfit. You will rarely train a large model from scratch: it is more common to use pretrained layers of state-of-the-art models.

## Number of  Neurons per Hidden Layer

It is common to start with bigger number and decrease gradually over layers. If your model is very deep first increase then decrease the number of neurons. 

You can start with more layers and neurons than your model need, then perform regularization and early stopping techniques to prevent overfitting: this is called "stretch pants" approach.

> In general you will get more better performance by increasing number of layers instead of the number of neurons.

## Learning Rate

One way to find start with small $10^{-5}$ and increase gradually up to 0.1 over some iterations. You'll see loss first decrease and then increasing, choose minimum point and start training with that.

After that you can decrease learning rate as epoch increases to converge better, this is called *learning schedule*.

## Optimizer

Generally we use Adam as default optimizer, or you can choose according to your datasets. Also tuning it's hyperparameters are important. We'll see advanced optimizer on further tutorials.

## Batch Size

These are set in 2 to the power as memories are assigned in that number. From research papers we can say that you could use large batch size with learning rate warm up (small to large), and if results are not good then use smaller 32-64 batch size instead.

> Learning rate is sensitive to other hyperparameters, so if you change any other hyperparameter make sure to tune learning rate also.

For more best practices you can check this excellent paper [A disciplined approach to neural network hyper-parameters](https://homl.info/1cycle).

